<a href="https://colab.research.google.com/github/vuchicago/Python/blob/master/Oil%26Gas_Effects.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
####MOUNT DRIVE
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)





Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os
os.chdir("/content/gdrive/My Drive/Python/Oil_Gas_MSA")
import pandas as pd
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from datetime import datetime as dt

base_dir="/content/gdrive/My Drive/Python/Oil_Gas_MSA"

pred_var=['index',"Private industries","Forestry,fishing","Oil_gas",'Non-metal miner manufac']

input_files=os.listdir(base_dir)

In [0]:
msa_files=pd.DataFrame()
for file in input_files:
    msa_file=pd.read_csv(file)
    keep_indices=[0,1,4,5,9,10,14,34,50,56,65,78]
    column_keep=["Description", "2001","2002",'2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011',
       '2012', '2013', '2014', '2015', '2016', '2017']
    msa_file=msa_file[column_keep].iloc[keep_indices,:]
    msa_file_T=pd.DataFrame(msa_file.T[1:]).reset_index()
    msa_file_T=msa_file_T.rename(columns={1:"Private industries",4:"Forestry,fishing",5:"Oil_gas",9:"Utilities",
                                          10:"Manufacturing",14:"Non-metal miner manufac",34:"Retail",
                                          50:"Finance_Ins",56:"Real_estate",65:"Education",78:"Government"})*1
    for var in msa_file_T.columns:
        msa_file_T[var]=msa_file_T[var].replace("(D)",np.NaN).replace("(NA)",np.nan).replace("(L)",np.nan)*1
        msa_file_T[var]=msa_file_T[var].apply(lambda x: np.nan if isinstance(np.float(x),str) else x)
        msa_file_T[var]=msa_file_T[var].fillna(method='ffill')
        msa_file_T[var]=msa_file_T[var].fillna(method='bfill')
        msa_file_T[var]=msa_file_T[var].replace(np.nan,0)
    #for var in msa_file_T.columns:
    msa_files=pd.concat([msa_files,msa_file_T],axis=0)





In [0]:
def feature_engineering(dset_in):
    dset_out=pd.DataFrame()
    for col in dset_in.columns:
        dset_out[col]=dset_in[col]
        dset_out[col+"_log"]=dset_in[col].apply(lambda x: np.log(x+1))
        dset_out[col+"_squared"]=dset_in[col].apply(lambda x: np.square(x+1))
    return dset_out


In [0]:
msa_files["Recession_indicator"]=0
recession_years=['2001','2008','2009']
for i in range(len(msa_files)):
    if msa_files["index"].iloc[i] in recession_years:
        msa_files["Recession_indicator"].iloc[i]=1
    else:
        msa_files["Recession_indicator"].iloc[i]=0
        
pred_var=["Private industries","Forestry,fishing","Oil_gas",'Non-metal miner manufac',"Utilities","Manufacturing","Retail","Finance_Ins","Real_estate","Education","Government"]
label_var=["Recession_indicator"]
df_var=msa_files[pred_var]
df_label=msa_files[label_var]

####convert it to numeric matrix
df_var_mat=np.matrix(df_var,float)
df_label_mat=np.matrix(df_label,float)



/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [0]:
#####Sklearn doesn't return p-values of coefficients
from sklearn.linear_model import LogisticRegression
import sklearn.feature_selection
clf=LogisticRegression(random_state=0,solver='lbfgs').fit(df_var,df_label)
p_value=sklearn.feature_selection.f_regression(df_var,df_label)
clf.coef_



In [0]:
#####using statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.discrete import discrete_model


discrete_model.Logit(df_label_mat,df_var_mat)
results=sm.Logit(df_label_mat,df_var_mat)
results_f=results.fit()
print(results_f.summary())